In [1]:
import os
import pickle
import numpy as np
from glob import glob
import hist
from hist import Hist
from hist.axis import IntCategory, Regular, StrCategory
import mplhep as hep
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from collections import defaultdict
import itertools
from iminuit import Minuit
from iminuit.cost import LeastSquares

hep.style.use('CMS')

### Find files

In [2]:
def find_pickles(base_dir):
    """Return sorted list of all pickle files under base_dir recursively."""
    return sorted(glob(os.path.join(base_dir, "**/*.pickle"), recursive=True))

### Combine pickles per config+method+category

In [3]:
def combine_pickles_per_config(file_list):
    """
    Combine pickles with common method+reco (bin-by-bin). 
    Extract 'method' and 'reco' from the filename.
    Example file: "postee_hist__var_phi_cp_mu_a1_3pr_dp_reco.pickle"
    Returns a dict: config_string -> combined histogram
    """
    groups = defaultdict(list)
    for f in file_list:
        name = os.path.basename(f)
        # split name after the first 'hist__' part
        parts = name.split("_")
        # extract method and reco (assume last two elements before .pickle)
        method, reco = parts[-2], parts[-1].replace(".pickle","")
        config_string = f"{method} {reco}"
        groups[config_string].append(f)

    combined = {}
    for config_string, files in groups.items():
        h = None
        for fpath in files:
            with open(fpath, "rb") as f:
                hh = pickle.load(f)
            h = hh.copy() if h is None else h + hh
        combined[config_string] = h

    return combined

### Extract histograms for individual categories

In [4]:
def extract_category(histogram, category):
    """Return histogram restricted to a single category."""
    print(f"category", category)
    return histogram[{"category": category}]

### Map processes for CP hypotheses

In [64]:
def map_processes_to_cp(process_axis, process_map, hist, cfg_name=None):
    """
    Return dictionary with CP hypothesis arrays.
    cp_arrays[cp_key] = {"values": ..., "errors": ..., "method": ..., "reco": ...}
    """
    cp = {}
    for proc, cp_key in process_map.items():
        if proc not in process_axis:
            continue
        h = hist[{"process": proc}]

        # Extract method & reco from cfg_name if given
        method = "unknown"
        reco   = "unknown"
        if cfg_name:
            parts = cfg_name.split()
            if parts[0] in ("ggF", "VBF"):
                parts = parts[1:]
            if len(parts) >= 2:
                method = parts[0].lower()  # z.B. dp, pv
                reco   = parts[1].lower()  # z.B. reco, mtt, gef

        cp[cp_key] = {
            "values": np.ravel(h.values()),
            "errors": np.ravel(h.variances() ** 0.5),
            "method": method,
            "reco": reco
        }

    return cp

### Fit and Asymmetry Functions

In [6]:
def model(x, a, b, c):
    return a*np.cos(x+c) + b

def fit(x, y, err=0.05, model=model):
    """Fit data with model and return Minuit object and parameters."""
    lsq = LeastSquares(x, y, err, model)
    m = Minuit(lsq, a=0.1, b=0.1, c=1.0)
    m.fixed = False
    m.migrad()
    m.hesse()
    return m, err, m.values["a"], m.errors["a"], m.values["b"], m.errors["b"], m.values["c"]

def comp_asymmetry(arr1, arr2):
    return (1/arr1.size)*np.sum(np.abs((arr1-arr2)/(arr1+arr2)))

def comp_asymmetry_error(arr1, arr2, err1, err2):
    denom = arr1 + arr2
    term1 = err1 * np.abs((2 * arr2) / (denom**2))
    term2 = err2 * np.abs((2 * arr1) / (denom**2))
    sigma_A = np.sqrt(np.sum(term1**2 + term2**2)) / arr1.size
    return sigma_A

### Plotting

In [80]:
def plot_phicp(cp_arrays, x, outpath, title, category_label, shiftdict=None):
    plt.figure(figsize=(8.9, 6.6))
    hep.cms.text("Private Work", loc=1)
    line_width = 2
    legend_line_width = line_width

    for cp_key, arr in cp_arrays.items():
        if cp_key == "asymmetry": 
            continue
        vals = arr["values"]
        errs = np.array(arr["errors"])
        errs[errs == 0] = 1e-6

        colour = shiftdict[cp_key]["colour"] if shiftdict else "black"
        linestyle = shiftdict[cp_key]["linestyle"] if shiftdict else "solid"
        location = shiftdict[cp_key]["location"] if shiftdict else "upper right"

        plt.errorbar(x, vals, errs, fmt="o", color=colour)
        m, err, a, a_err, b, b_err, c = fit(x, vals, errs)
        fit_curve = model(x, a, b, c)
        plt.plot(x, fit_curve, color=colour, linestyle=linestyle, linewidth=line_width)

        fit_info = [f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} ≈ {m.fmin.reduced_chi2:.1f}"]
        legend_handle = Line2D([0], [0], color=colour, linestyle=linestyle, linewidth=legend_line_width, label=f"CP {cp_key[3:]}")
        legend = plt.legend(handles=[legend_handle], title="\n".join(fit_info), frameon=False, loc=location, fontsize=15, title_fontsize=12)
        plt.gca().add_artist(legend)

        y_min = np.min(fit_curve)*0.7
        y_max = np.max(fit_curve)*1.2
        plt.ylim(y_min, y_max)

    # X-axis: cfg_name (method + Reco)
    cfg_names = [arr.get("method", "unk") + " " + arr.get("reco", "unk") for arr in cp_arrays.values() if isinstance(arr, dict) and "values" in arr]
    x_label = r"$\Phi_{CP}$" + (f" ({cfg_names[0]})" if cfg_names else "")
    plt.xlabel(x_label)
    plt.ylabel("a.u.")

    # infobox
    #info_text = f"category: {category_label}"

    info_text_lines = []

    if simpledict:
        # Original-Key finden
        reverse_simple = {v["simple"]: k for k, v in simpledict.items()}
        original_key = reverse_simple.get(category_label, category_label)
        math_label = simpledict.get(original_key, {}).get("math", category_label)
        info_text_lines.append(f"Category: {math_label}")
        info_text = "\n".join(info_text_lines)

    plt.annotate(info_text, xy=(0.01, 1.012), xycoords="axes fraction", fontsize=14,
                 bbox=dict(boxstyle="round,pad=0.3", facecolor='none', edgecolor='none'))

    # Title with asymmetry
    asymmetry = cp_arrays.get("asymmetry", None)
    if asymmetry is not None and "even_odd" in asymmetry:
        a_val = asymmetry["even_odd"]["asymmetry_val"]
        a_err = asymmetry["even_odd"]["asymmetry_error"]
        plt.title(f"($A_{{even,odd}}$ = {a_val:.3f} ± {a_err:.3f})", fontsize=25, loc='right')
    else:
        plt.title(title, fontsize=25, loc='right')

    plt.tight_layout()
    os.makedirs(os.path.dirname(outpath), exist_ok=True)
    plt.savefig(outpath, dpi=300, bbox_inches='tight')
    plt.close()


In [81]:
def compare_asymmetries(cp_arrays, category_label, selected_keys=None, base=None):
    """
    Compare asymmetries for ggF and VBF side by side for each (method, reco) combination.
    Sorted: method (dp -> pv), reco (gen -> reco -> mtt -> gef)
    """
    if selected_keys is None:
        selected_keys = ['even_odd']

    # Output path
    if base is None:
        outdir = os.path.join("OUTPUT", "compare_asymmetry")
    else:
        relpath = os.path.relpath(base, "INPUT")
        first_part = relpath.split(os.sep)[0]
        outdir = os.path.join("OUTPUT", first_part)
    os.makedirs(outdir, exist_ok=True)
    outpath = os.path.join(outdir, f"Asymmetry_{category_label}_comparison.pdf")

    plt.figure(figsize=(7.7, 5.5))
    hep.cms.text("Private work", loc=0)

    # Group by (method, reco) combinations, regardless of production
    grouped = {}
    for cfg_string, cp_arrays in cp_arrays.items():
        # Extract production and method/reco
        if 'VBF' in cfg_string:
            prod = 'VBF'
            combo = cfg_string.replace('VBF', '').strip()
        else:
            prod = 'ggF'
            combo = cfg_string.replace('ggF', '').strip()

        if combo not in grouped:
            grouped[combo] = {}
        grouped[combo][prod] = cp_arrays

    # Sort by method/reconstruction
    def sort_key(combo):
        words = combo.lower().split()
        method = words[0] if words else ''
        reco   = words[1] if len(words) > 1 else ''
        return (method_order.index(method) if method in method_order else 99,
                reco_order.index(reco) if reco in reco_order else 99)

    sorted_combos = sorted(grouped.keys(), key=sort_key)

    width = 0.19
    gap = 0.02
    x_base = np.arange(len(sorted_combos))

    # Bar plot
    for i, combo in enumerate(sorted_combos):
        prod_dict = grouped[combo]
        for prod, cp_arrays in prod_dict.items():
            for key in selected_keys:
                if 'asymmetry' not in cp_arrays or key not in cp_arrays['asymmetry']:
                    continue
                entry = cp_arrays['asymmetry'][key]
                value = entry['asymmetry_val']
                error = entry['asymmetry_error']

                xpos = x_base[i] + (0 if prod == 'ggF' else width + gap)
                plt.bar(xpos, value, yerr=error, width=width, capsize=5,
                        color=colour_map[prod],
                        label=prod if i == 0 else "")

    # X-Ticks in the middle between ggF and VBF
    xticks = x_base + width/2 + gap/2
    xtick_labels = sorted_combos
    plt.xticks(xticks, xtick_labels, rotation=45)

    # adjust y-axis
    all_values = []
    for combo in sorted_combos:
        prod_dict = grouped[combo]
        for cp_arrays in prod_dict.values():
            for key in selected_keys:
                if 'asymmetry' in cp_arrays and key in cp_arrays['asymmetry']:
                    entry = cp_arrays['asymmetry'][key]
                    all_values.append(entry['asymmetry_val'])


    if all_values:  # if there are values
        y_min = 0
        y_max = max(all_values) * 1.4
        plt.ylim(y_min, y_max)


    plt.margins(x=0.1)
    plt.ylabel("Asymmetry")
    plt.legend(loc='upper right')

    # Infobox : Kategorie (Mathematik-Label) + optional Beispiel-Asymmetrie
    info_text_lines = []
    print(category_label)
    if simpledict:
        # Search for original key
        reverse_simple = {v["simple"]: k for k, v in simpledict.items()}
        original_key = reverse_simple.get(category_label, category_label)
        math_label = simpledict.get(original_key, {}).get("math", category_label)
        info_text_lines.append(f"Category: {math_label}")

    # Asymmetry key (e.g. 'even_odd')
    example_cp_arrays = next(iter(cp_arrays.values()))
    if 'asymmetry' in example_cp_arrays:
        asym_key = next(iter(example_cp_arrays['asymmetry'].keys()))
        # optional: formatiere schöner
        asym_label = " vs ".join(asym_key.split('_'))
        info_text_lines.append(f"Asymmetry of: CP {asym_label}")

    if info_text_lines:
        info_text = "\n".join(info_text_lines)
        plt.annotate(info_text, xy=(0.04, 0.94), xycoords="axes fraction", fontsize=12,
                     ha='left', va='top',
                     bbox=dict(boxstyle="round,pad=0.3", facecolor='white', edgecolor='black'))

    plt.grid(True)
    #plt.tight_layout()
    plt.savefig(outpath, dpi=300, bbox_inches='tight')
    plt.close()

## Main

In [82]:
method_order = ['dp', 'pv']
reco_order   = ['gen', 'reco', 'mtt', 'gef']

# Base directories
base_dirs = ["INPUT/05_DesyTau_fix_MTT/ggF",
             "INPUT/05_DesyTau_fix_MTT/VBF"]

# CP process maps
ggf_map = {
    "h_ggf_htt_sm_prod_sm": "cp_even",
    "h_ggf_htt_mm_prod_sm": "cp_maxmix",
    "h_ggf_htt_cpo_prod_sm": "cp_odd"
}
vbf_map = {
    "h_vbf_htt_sm": "cp_even",
    "h_vbf_htt_mm": "cp_maxmix",
    "h_vbf_htt_cpo": "cp_odd"
}

# Shiftdict for colours & linestyles
shiftdict = {
    "cp_even": {"colour": "#d62839", "location": "upper right", "linestyle": "solid"},
    "cp_odd": {"colour": "#28348e", "location": "lower left", "linestyle": (0, (2,2))},
    "cp_maxmix": {"colour": "#2b663c", "location": "lower right", "linestyle": (0, (2,1,0.5,1,0.5,1,0.5,1))}
}

simpledict = {
    'cat_mutau_sr' : {'simple' : 'mutau_inclusive', 'math' : r'$\mu \tau$ inclusive'},
    'cat_mutau_sr__tau2a1_3pr' : {'simple' : 'mua13pr', 'math' : r'$\mu a^{3pr}$'},
    'cat_mutau_sr__prompt__hig__cat2__tau2a1_3pr' : {'simple' : 'mua13pr_bdt', 'math' : r'$\mu a^{3pr}$ BDT signal'},
    'cat_mutau_sr__jet_fakes__fake' : {'simple' : 'cat_mutau_sr__jet_fakes__fake', 'math' : r'cat_mutau_sr__jet_fakes__fake'},
}

categories = list(simpledict.keys())
#categories = ['cat_mutau_sr__tau2a1_3pr','cat_mutau_sr']

colour_map = {'ggF': '#d62839', 'VBF': '#28348e', 'green':'#2b663c'}

In [83]:
# Storage for analysis
data_storage = defaultdict(dict)

In [84]:
for base in base_dirs:
    all_pickles = find_pickles(base)
    print(f"{base} found pickles: {len(all_pickles)}")

    combined_histograms = combine_pickles_per_config(all_pickles)
    print(f"{os.path.basename(base)}")

    # determine Higgs production mode
    if "VBF" in base:
        prod_str = "VBF"
        process_map = vbf_map
    else:
        prod_str = "ggF"
        process_map = ggf_map

    for category in categories:
        if category not in cp_storage:
            cp_storage[category] = {}
        if category not in data_storage:
            data_storage[category] = {}

        for cfg_name, hist_obj in combined_histograms.items():
            hcat = extract_category(hist_obj, category)
            #print(f"{cfg_name}: sum={hcat.sum()}")

            # Calculate CP values for this category and cfg_name
            cp_arrays = map_processes_to_cp(
                hcat.axes['process'],
                process_map,
                hcat,
                cfg_name=cfg_name
            )

            # Calculate asymmetry if cp_even and cp_odd exist
            if 'cp_even' in cp_arrays and 'cp_odd' in cp_arrays:
                vals_even = cp_arrays['cp_even']['values']
                vals_odd  = cp_arrays['cp_odd']['values']
                errs_even = cp_arrays['cp_even']['errors']
                errs_odd  = cp_arrays['cp_odd']['errors']

                asym_val = comp_asymmetry(vals_even, vals_odd)
                asym_err = comp_asymmetry_error(vals_even, vals_odd, errs_even, errs_odd)

                cp_arrays['asymmetry'] = {
                    'even_odd': {'asymmetry_val': asym_val, 'asymmetry_error': asym_err}
                }

            # Key for storage: Production + cfg_name
            key = f"{prod_str} {cfg_name}"

            cp_storage[category][key] = cp_arrays
            data_storage[category][key] = cp_arrays

            # OUTPUT path for PhiCP
            relpath = os.path.relpath(base, "INPUT")
            outdir = os.path.join("OUTPUT", relpath)
            os.makedirs(outdir, exist_ok=True)
            outname = f"{outdir}/phi_{category}_{cfg_name}.pdf"  # nur cfg_name im Titel

            # Plot PhiCP
            x = np.linspace(0, 2*np.pi, hcat.shape[-1])
            plot_phicp(cp_arrays, x, outname, title=cfg_name, category_label=category, shiftdict=shiftdict)

# --- Asymmetry Comparison ---
for category, category_cp_arrays in cp_storage.items():
    compare_asymmetries(
        cp_storage=category_cp_arrays,
        category_label=category,
        selected_keys=['even_odd'],
        base=base
    )
    print("bar plot ready for ", category)

INPUT/05_DesyTau_fix_MTT/ggF found pickles: 48
ggF
category cat_mutau_sr
category cat_mutau_sr
category cat_mutau_sr
category cat_mutau_sr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__prompt__hig__cat2__tau2a1_3pr
category cat_mutau_sr__prompt__hig__cat2__tau2a1_3pr
category cat_mutau_sr__prompt__hig__cat2__tau2a1_3pr
category cat_mutau_sr__prompt__hig__cat2__tau2a1_3pr
category cat_mutau_sr__jet_fakes__fake
category cat_mutau_sr__jet_fakes__fake
category cat_mutau_sr__jet_fakes__fake
category cat_mutau_sr__jet_fakes__fake
INPUT/05_DesyTau_fix_MTT/VBF found pickles: 48
VBF
category cat_mutau_sr
category cat_mutau_sr
category cat_mutau_sr
category cat_mutau_sr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__tau2a1_3pr
category cat_mutau_sr__prompt__hig__cat2__tau2a1_3pr
category cat_mutau_sr__prompt

TypeError: compare_asymmetries() got an unexpected keyword argument 'cp_storage'

### Exemplary sliced histogram

### exemplary sliced histogram

In [ ]:
def plot_phicp(cp_arrays, x, outpath, title, category_label, shiftdict=None):
    plt.figure(figsize=(8.9, 6.6))
    hep.cms.text("Private Work", loc=1)
    line_width = 2
    legend_line_width = line_width

    for cp_key, arr in cp_arrays.items():
        if cp_key == "asymmetry": 
            continue
        vals = arr["values"]
        errs = np.array(arr["errors"])
        errs[errs == 0] = 1e-6

        colour = shiftdict[cp_key]["colour"] if shiftdict else "black"
        linestyle = shiftdict[cp_key]["linestyle"] if shiftdict else "solid"
        location = shiftdict[cp_key]["location"] if shiftdict else "upper right"

        plt.errorbar(x, vals, errs, fmt="o", color=colour)
        m, err, a, a_err, b, b_err, c = fit(x, vals, errs)
        fit_curve = model(x, a, b, c)
        plt.plot(x, fit_curve, color=colour, linestyle=linestyle, linewidth=line_width)

        fit_info = [f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} ≈ {m.fmin.reduced_chi2:.1f}"]
        legend_handle = Line2D([0], [0], color=colour, linestyle=linestyle, linewidth=legend_line_width, label=f"CP {cp_key[3:]}")
        legend = plt.legend(handles=[legend_handle], title="\n".join(fit_info), frameon=False, loc=location, fontsize=15, title_fontsize=12)
        plt.gca().add_artist(legend)

        y_min = np.min(fit_curve)*0.7
        y_max = np.max(fit_curve)*1.2
        plt.ylim(y_min, y_max)

    # X-axis: cfg_name (method + Reco)
    cfg_names = [arr.get("method", "unk") + " " + arr.get("reco", "unk") for arr in cp_arrays.values() if isinstance(arr, dict) and "values" in arr]
    x_label = r"$\Phi_{CP}$" + (f" ({cfg_names[0]})" if cfg_names else "")
    plt.xlabel(x_label)
    plt.ylabel("a.u.")

    # infobox
    #info_text = f"category: {category_label}"

    info_text_lines = []

    if simpledict:
        # Original-Key finden
        reverse_simple = {v["simple"]: k for k, v in simpledict.items()}
        original_key = reverse_simple.get(category_label, category_label)
        math_label = simpledict.get(original_key, {}).get("math", category_label)
        info_text_lines.append(f"Category: {math_label}")
        info_text = "\n".join(info_text_lines)

    plt.annotate(info_text, xy=(0.01, 1.012), xycoords="axes fraction", fontsize=14,
                 bbox=dict(boxstyle="round,pad=0.3", facecolor='none', edgecolor='none'))

    # Title with asymmetry
    asymmetry = cp_arrays.get("asymmetry", None)
    if asymmetry is not None and "even_odd" in asymmetry:
        a_val = asymmetry["even_odd"]["asymmetry_val"]
        a_err = asymmetry["even_odd"]["asymmetry_error"]
        plt.title(f"($A_{{even,odd}}$ = {a_val:.3f} ± {a_err:.3f})", fontsize=25, loc='right')
    else:
        plt.title(title, fontsize=25, loc='right')

    plt.tight_layout()
    os.makedirs(os.path.dirname(outpath), exist_ok=True)
    plt.savefig(outpath, dpi=300)
    plt.close()
